In [1]:
import importlib
cim_profile = 'rc4_2021'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

In [2]:
from cimgraph.databases import ConnectionParameters
from cimgraph.databases.blazegraph import BlazegraphConnection
from cimgraph.models import FeederModel
from cimgraph.models.graph_model import new_mrid

In [3]:
feeder_mrid = "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
feeder = cim.Feeder(mRID=feeder_mrid)

In [4]:
# Blazegraph connection for running outside the container
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql",
                              cim_profile='rc4_2021', iec61970_301=7)
bg = BlazegraphConnection(params)


In [5]:
network = FeederModel(connection=bg, container=feeder, distributed=False)


In [6]:
network.pprint(cim.PowerElectronicsConnection)

{}


In [7]:
new_inv = cim.PowerElectronicsConnection(mRID=new_mrid(), name = "new_inv")

In [8]:
new_inv.EquipmentContainer = feeder

In [14]:
new_inv.ratedS = 2000
new_inv.ratedU = 120
new_inv.inverterMode = cim.ConverterControlModeKind.constantPowerFactor

AttributeError: module 'cimgraph.data_profile.rc4_2021' has no attribute 'ConverterControlModeKind'

In [10]:
network.add_to_graph(new_inv)

In [11]:
network.pprint(cim.PowerElectronicsConnection)

{
    "3ac7658a-f657-41e7-867e-8fc4cbdffc31": {
        "mRID": "3ac7658a-f657-41e7-867e-8fc4cbdffc31",
        "name": "new_inv",
        "EquipmentContainer": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "ratedS": "2000",
        "ratedU": "120"
    }
}


In [12]:
total_p = 0
network.get_all_edges(cim.PowerElectronicsConnection)
network.get_all_edges(cim.PhotovoltaicUnit)
for pv in network.graph[cim.PhotovoltaicUnit].values():
    total_p = total_p + float(pv.maxP)

print(total_p)

KeyError: <class 'cimgraph.data_profile.rc4_2021.gridappsd_cim_profile_rc4_2021.PhotovoltaicUnit'>

In [ ]:

for line in network.graph[cim.ACLineSegment].values():
    for phs in line.phases:
        if phs.phases == "A":
            total_a_length += line.length

In [13]:
network.upload()

/home/ande188/.local/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


In [ ]:
print("""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\n        PREFIX cim: <http://iec.ch/TC57/CIM100#>\n        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n         INSERT DATA { \n        <http://localhost:8889/bigdata/namespace/kb/sparql#_8d5aef93-928a-479d-a706-8038aea11278> a cim:PowerElectronicsConnection.\n                \n        <http://localhost:8889/bigdata/namespace/kb/sparql#_8d5aef93-928a-479d-a706-8038aea11278> cim:PowerElectronicsConnection.ratedS "2000".\n                                     \n        <http://localhost:8889/bigdata/namespace/kb/sparql#_8d5aef93-928a-479d-a706-8038aea11278> cim:PowerElectronicsConnection.ratedU "120".\n                                     \n        <http://localhost:8889/bigdata/namespace/kb/sparql#_8d5aef93-928a-479d-a706-8038aea11278> cim:Equipment.EquipmentContainer <rdf:resource="#49AD8E07-3BF9-A4E2-CB8F-C3722F837B62">.\n                                        \n        <http://localhost:8889/bigdata/namespace/kb/sparql#_8d5aef93-928a-479d-a706-8038aea11278> cim:IdentifiedObject.mRID "8d5aef93-928a-479d-a706-8038aea11278".\n                                     \n        <http://localhost:8889/bigdata/namespace/kb/sparql#_8d5aef93-928a-479d-a706-8038aea11278> cim:IdentifiedObject.name "new_inv".\n                                     }""")